In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql("SET spark.sql.shuffle.partitions = 400;")
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment AS
WITH main AS (
    SELECT 
        store_id,
        product_id, 
        dr,
        note,
        cluster asst,
        start_date,
        end_date
    FROM (
        SELECT 
            a.*, 
            ROW_NUMBER() OVER (PARTITION BY store_id, product_id, cluster, end_date ORDER BY proc_date) rn 
        FROM (
            (SELECT 
                b.store_id, product_id, dr, note, cluster_asst cluster, proc_date, start_date, end_date
            FROM 
                 (select /*+ REPARTITION(1000) */ product_id, dr, note, cluster_asst , proc_date, start_date, end_date from {catalog_name}.udp_wcm_silver_dim.stg_vmp_assortmentmb a) a
                CROSS JOIN ( select b.store_id from {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b where b.region_domain = 'North' AND business_unit = '2000') b
                )
            UNION ALL 
            (SELECT
                b.store_id, product_id, dr,
                CASE 
                    WHEN LOWER(a.region) RLIKE 'đà nẵng$|quảng nam$|quảng ngãi$|quảng trị$|thừa thiên huế$' THEN note_dng
                ELSE note
            END AS note,
                            cluster_asst cluster, proc_date, start_date, end_date
            FROM 
                ( select/*+ REPARTITION(1000) */ product_id, dr, region,note_dng, note, cluster_asst, proc_date, start_date, end_date from  {catalog_name}.udp_wcm_silver_dim.stg_vmp_assortmentmn a ) a
                CROSS JOIN  ( select b.store_id from {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b where b.region_domain <> 'North' AND business_unit = '2000') b
                )
            UNION ALL 
            (SELECT 
                store_id, product_id, 'Có tính' dr, NULL note, NULL cluster, proc_date, start_date, end_date
            FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_assortmentmn
            )
            UNION ALL 
            (SELECT 
                store_id, product_id, 'Có tính' dr, NULL note, NULL cluster, proc_date, start_date, end_date
            FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_assortmentmb a
            )
        ) a 
    ) x WHERE x.rn = 1
) 
SELECT 
    *, LAG(start_date,1) OVER (PARTITION BY store_id,product_id ORDER BY start_date) day_of_sale 
FROM main;
""")